# Writing Fast Code

The first thing to know about writing fast code is that fast code generally comes **after** making code work.

One common saying is *"Premature optimization is the root of all evil"* because by trying to make code fast before desining it well, you make everything harder. Here a good pattern for development:

1. Decide how fast your code should need to be

2. Decide on the tools libraries necessary to achieve that

3. Make it work

4. Make it fast. Do this by improving the code and **benchmarking** it. Start with the "hot loop" that gets run the most, then expand to optimizing more code as necessary.

5. Make it easy to read

The reason we can't simply "make it work" $\rightarrow$ "make it fast" is that if your first design uses libraries that can't fundamentally work at the scale you need might make it hard to change to a faster design afterwards.

That said, making your code work is the most important. Making code fast is rarely a necessity as much as delivering working code.

# Fast code: a recap

The main way to write fast code is to have code be packed tightly in memory so movements between memory systems (CPU registers, CPU cache, RAM, disk drives, internet) are minimized. For reference, here is a handy chart of time taken on a normal x86 CPU to fetch data from different places:

```
Latency Comparison Numbers (~2012)
----------------------------------
L1 cache reference                           0.5 ns
Branch mispredict                            5   ns
L2 cache reference                           7   ns                      14x L1 cache
Mutex lock/unlock                           25   ns
Main memory reference                      100   ns                      20x L2 cache, 200x L1 cache
Compress 1K bytes with Zippy             3,000   ns        3 us
Send 1K bytes over 1 Gbps network       10,000   ns       10 us
Read 4K randomly from SSD*             150,000   ns      150 us          ~1GB/sec SSD
Read 1 MB sequentially from memory     250,000   ns      250 us
Round trip within same datacenter      500,000   ns      500 us
Read 1 MB sequentially from SSD*     1,000,000   ns    1,000 us    1 ms  ~1GB/sec SSD, 4X memory
Disk seek                           10,000,000   ns   10,000 us   10 ms  20x datacenter roundtrip
Read 1 MB sequentially from disk    20,000,000   ns   20,000 us   20 ms  80x memory, 20X SSD
Send packet CA->Netherlands->CA    150,000,000   ns  150,000 us  150 ms
```

# Numpy and compiled functions

As we saw in the numpy lecture, numpy is fast because arrays are packed in memory and the numpy library functions are written in compiled C code, instead of interpreted python code. 

Python code has to be re-compiled every time you run it. In a loop it'll re-compile on every iteration. 

The best way to start optimizing code is to rewrite it to use numpy, pandas or scipy functions instead of python code.


# Benchmarking

The fastest way to start benchmarking your code is to use the `%timeit` command:

In [1]:
n = 100000

%timeit sum([1. / i**2 for i in range(1, n)])

31.5 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
import numpy as np

%timeit np.sum(1. / np.arange(1., n) ** 2)

210 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Memory Layout

In [3]:
a = np.random.rand(100_000_000).reshape((10_000, 10_000))

In [4]:
%timeit a.sum(axis=0)

76.2 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit a.sum(axis=1)

84.4 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Summing the array row-wise instead of column-wise is 150% faster, because the data is stored row-by-row.

So when iterating through columns, we miss the CPU cache more often

In the Pandas library, this is reversed (data is stored by column)

# Profiling

On top of benchmarking parts of the code, you can **profile** the code, telling us how much time is spent on each line

In [6]:
# If line_profiler isn't installed, install it
# !pip install line_profiler

In [7]:
%load_ext line_profiler

In [8]:
with open('simulation.py', 'w') as f:
    f.write("""
import numpy as np

def step(*shape):
    # Create a random n-vector with +1 or -1 values.
    return 2 * (np.random.random_sample(shape)<.5) - 1

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                   for i in range(iterations)])
    return y
""")

In [9]:
from simulation import simulate

%lprun -T lprof0 -f simulate simulate(50)


*** Profile printout saved to text file 'lprof0'. 


# Profiling RAM usage

In [10]:
# If memory_profiler isn't installed, install it
# !pip install memory_profiler

In [11]:
%load_ext memory_profiler

In [12]:
with open('memscript.py', 'w') as f:
    f.write("""
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a
""")

In [13]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()



*** Profile printout saved to text file mprof0. 
